# AlexNet implementation example using TensorFlow

[paper](http://papers.nips.cc/paper/4824-imagenet-classification-with-deep-convolutional-neural-networks.pdf)

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

## Import Mnist dataset

In [2]:
mnist = input_data.read_data_sets('/tmp/data/', one_hot=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


## Parameters

In [3]:
learning_rate = 0.001
training_iters = 300000
batch_size = 64
display_step = 100

## Network parameters

In [4]:
n_input = 784 # MNIST data input (img shape = 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)
dropout = 0.8 # Dropout, probability to keep units

## tf Graph input

In [5]:
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32) # dropout, keep probability

## Create AlexNet model

In [6]:
def conv2d(name, l_input, w, b):
    return tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(l_input, 
                                                  w, 
                                                  strides=[1, 1, 1, 1],
                                                  padding='SAME'), 
                                     b,
                                     name=name))

In [18]:
def max_pool(name, l_input, k):
    return tf.nn.max_pool(l_input,
                          ksize=[1, k, k, 1],
                          strides=[1, k, k, 1],
                          padding='SAME',
                          name=name)

In [24]:
def norm(name, l_input, lsize=4):
    return tf.nn.lrn(l_input,
                     lsize,
                     bias=1.0,
                     alpha=0.001 / 9.,
                     beta=0.75,
                     name=name)

In [25]:
def alex_net(_X, _weights, _biases, _dropout):
    
    # Reshape input picture
    _X = tf.reshape(_X, [-1, 28, 28, 1])
    
    # Convolution layer
    conv1 = conv2d('conv1', _X, _weights['wc1'], _biases['bc1'])
    # Max Pooling (down-sampling)
    pool1 = max_pool('pool1', conv1, k=2)
    # Apply Normalization
    norm1 = norm('norm1', pool1, lsize=4)
    # Apply Dropout
    norm1 = tf.nn.dropout(norm1, _dropout)
    
    # Convolution layer
    conv2 = conv2d('conv2', norm1, _weights['wc2'], _biases['bc2'])
    # Max Pooling (down-sampling)
    pool2 = max_pool('pool2', conv2, k=2)
    # Apply Normalization
    norm2 = norm('norm2', pool2, lsize=4)
    # Apply Dropout
    norm2 = tf.nn.dropout(norm2, _dropout)
    
    # Convolution layer
    conv3 = conv2d('conv3', norm2, _weights['wc3'], _biases['bc3'])
    # Max Pooling (down-sampling)
    pool3 = max_pool('pool3', conv3, k=2)
    # Apply Normalization
    norm3 = norm('norm3', pool3, lsize=4)
    # Apply Dropout
    norm3 = tf.nn.dropout(norm3, _dropout)
    
    # Fully connected layer
    # Reshape conv3 output to fit dense layer input
    dense1 = tf.reshape(norm3, [-1, _weights['wd1'].get_shape().as_list()[0]])
    # Relu activation
    dense1 = tf.nn.relu(tf.matmul(dense1, _weights['wd1']) + _biases['bd1'], name='fc1')
    
    # Relu activation
    dense2 = tf.nn.relu(tf.matmul(dense1, _weights['wd2']) + _biases['bd2'], name='fc2')
    
    # Output, class prediction
    out = tf.matmul(dense2, _weights['out']) + _biases['out']
    
    return out

## Store layers' weights and biases

In [26]:
weights = {
    'wc1': tf.Variable(tf.random_normal([3, 3, 1, 64])),
    'wc2': tf.Variable(tf.random_normal([3, 3, 64, 128])),
    'wc3': tf.Variable(tf.random_normal([3, 3, 128, 256])),
    'wd1': tf.Variable(tf.random_normal([4*4*256, 1024])),
    'wd2': tf.Variable(tf.random_normal([1024, 1024])),
    'out': tf.Variable(tf.random_normal([1024, 10]))
}

In [29]:
biases = {
    'bc1': tf.Variable(tf.random_normal([64])),
    'bc2': tf.Variable(tf.random_normal([128])),
    'bc3': tf.Variable(tf.random_normal([256])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'bd2': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

## Construct the model

In [30]:
pred = alex_net(x, weights, biases, keep_prob)

## Define loss and optimizer

In [31]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

## Evaluate the model

In [32]:
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

## Initializing the variables

In [33]:
init = tf.initialize_all_variables()

## Launch the graph

In [34]:
with tf.Session() as sess:
    sess.run(init)
    step = 1
    
    while step * batch_size < training_iters:
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        
        # Fit training using batch data
        sess.run(optimizer, feed_dict={x: batch_x,
                                       y: batch_y,
                                       keep_prob: dropout})
        
        if step % display_step == 0:
            # Calculate batch accuracy
            acc = sess.run(accuracy, feed_dict={x: batch_x,
                                                y: batch_y,
                                                keep_prob: 1.})
            # Calculate batch loss
            loss = sess.run(cost, feed_dict={x: batch_x,
                                                y: batch_y,
                                                keep_prob: 1.})
            
            print("Iter {}, Minibatch loss= {:.6f}, Training Accuracy= {:.5f}".format(step * batch_size, loss, acc))
        
        step += 1
    
    print("Optimization finished!")
    # Calculate accuracy for 256 MNIST test images
    test_acc = sess.run(accuracy, feed_dict={x: mnist.test.images[:256],
                                             y: mnist.test.labels[:256],
                                             keep_prob: 1.})
    print("Testing Accuracy= {}".format(test_acc))

Iter 6400, Minibatch loss= 33809.773438, Training Accuracy= 0.56250
Iter 12800, Minibatch loss= 22527.335938, Training Accuracy= 0.62500
Iter 19200, Minibatch loss= 11973.410156, Training Accuracy= 0.67188
Iter 25600, Minibatch loss= 19344.394531, Training Accuracy= 0.65625
Iter 32000, Minibatch loss= 7425.503906, Training Accuracy= 0.75000
Iter 38400, Minibatch loss= 5462.512695, Training Accuracy= 0.79688
Iter 44800, Minibatch loss= 5546.448242, Training Accuracy= 0.79688
Iter 51200, Minibatch loss= 6364.197754, Training Accuracy= 0.82812
Iter 57600, Minibatch loss= 3692.698242, Training Accuracy= 0.85938
Iter 64000, Minibatch loss= 7789.284668, Training Accuracy= 0.84375
Iter 70400, Minibatch loss= 7346.293457, Training Accuracy= 0.79688
Iter 76800, Minibatch loss= 3944.713623, Training Accuracy= 0.82812
Iter 83200, Minibatch loss= 5050.075195, Training Accuracy= 0.85938
Iter 89600, Minibatch loss= 5345.741211, Training Accuracy= 0.87500
Iter 96000, Minibatch loss= 2710.572021, Trai